In [1]:
# 📦 Imports
import pandas as pd
import numpy as np
import joblib
import plotly.graph_objects as go
from sklearn.metrics import mean_squared_error
from pathlib import Path
from IPython.display import display

# 📁 Carregar dados com caminho absoluto seguro
ethanol_path = Path(r"C:/Users/Paulo Eduardo/Documents/Dissertação/ModelosML/Conect2ai/MDPI2023-pollution/data/[Etanol] Trajeto Casa-Escola-UFRN/trackLog-2023-Feb-13_06-38-49_seg.csv")
gasoline_path = Path(r"C:/Users/Paulo Eduardo/Documents/Dissertação/ModelosML/Conect2ai/MDPI2023-pollution/data/[Gasolina] Trajeto Casa-Escola-UFRN/trackLog-2022-Dec-01_06-43-57_qui.csv")

df_ethanol = pd.read_csv(ethanol_path)
df_gasoline = pd.read_csv(gasoline_path)

# ✨ Preparo dos dados
def preparar_dados(df):
    df.columns = df.columns.str.strip()
    col_map = {
        'Speed (OBD)(km/h)': 'Speed(OBD)(km/h)',
        'Latitude ': 'Latitude',
        'Longitude ': 'Longitude'
    }
    df.rename(columns={k: v for k, v in col_map.items() if k in df.columns}, inplace=True)
    if "Speed(OBD)(km/h)" in df.columns:
        speeds = df["Speed(OBD)(km/h)"].fillna(0).values * 1000 / 3600
        df["Acceleration"] = np.diff(speeds, prepend=speeds[0])
    return df

df_ethanol = preparar_dados(df_ethanol)
df_gasoline = preparar_dados(df_gasoline)

# ✨ Preparo dos dados
def preparar_dados(df):
    df.columns = df.columns.str.strip()
    col_map = {
        'Speed (OBD)(km/h)': 'Speed(OBD)(km/h)',
        'Latitude ': 'Latitude',
        'Longitude ': 'Longitude'
    }
    df.rename(columns={k: v for k, v in col_map.items() if k in df.columns}, inplace=True)
    if "Speed(OBD)(km/h)" in df.columns:
        speeds = df["Speed(OBD)(km/h)"].fillna(0).values * 1000 / 3600
        df["Acceleration"] = np.diff(speeds, prepend=speeds[0])
    return df

df_ethanol = preparar_dados(df_ethanol)
df_gasoline = preparar_dados(df_gasoline)

# 📦 Carregar modelos
modelos = {
    "ethanol": {
        "afr": joblib.load(Path(r"C:/Users/Paulo Eduardo/Documents/Dissertação/ModelosML/Conect2ai/MDPI2023-pollution/models/LGBMRegressor_ethanol_afr.pkl")),
        "maf": joblib.load(Path(r"C:/Users/Paulo Eduardo/Documents/Dissertação/ModelosML/Conect2ai/MDPI2023-pollution/models/XGBRegressor_ethanol_maf.pkl"))
    },
    "gasoline": {
        "afr": joblib.load(Path(r"C:/Users/Paulo Eduardo/Documents/Dissertação/ModelosML/Conect2ai/MDPI2023-pollution/models/LGBMRegressor_gasoline_afr.pkl")),
        "maf": joblib.load(Path(r"C:/Users/Paulo Eduardo/Documents/Dissertação/ModelosML/Conect2ai/MDPI2023-pollution/models/XGBRegressor_gasoline_maf.pkl"))
    }
}

# 🔧 Função compute_co2
def compute_co2(afr, maf, fuel_type="ethanol"):
    carbon_fraction = {
        "ethanol": 0.5217,
        "gasoline": 0.8571
    }
    c_fraction = carbon_fraction[fuel_type]
    fuel_rate = maf / afr
    return fuel_rate * c_fraction * (44.0 / 12.0)

# 🔧 Função auxiliar para encontrar colunas reais de AFR e MAF
def encontrar_coluna(df, alternativas):
    df.columns = df.columns.str.strip()
    for alt in alternativas:
        if alt in df.columns:
            return df[alt]
    return None

# 🔍 Função principal para executar os testes com visualização Plotly e cálculo de métricas
def executar_teste_plotly(df_original, df_modificado, modelo_afr, modelo_maf, fuel, nome_mr):
    features = ["Latitude", "Longitude", "Speed(OBD)(km/h)", "Acceleration"]

    afr_real = encontrar_coluna(df_original, ["Air Fuel Ratio(Commanded)(:1)", "AFR", "AirFuelRatio(Commanded)(:1)"])
    maf_real = encontrar_coluna(df_original, ["Mass Air Flow Rate(g/s)", "MAF", "MassAirFlowRate(g/s)"])

    co2_real = compute_co2(afr_real.values, maf_real.values, fuel) if afr_real is not None and maf_real is not None else None

    afr_orig = modelo_afr.predict(df_original[features])
    maf_orig = modelo_maf.predict(df_original[features])
    co2_orig = compute_co2(afr_orig, maf_orig, fuel)

    afr_mod = modelo_afr.predict(df_modificado[features])
    maf_mod = modelo_maf.predict(df_modificado[features])
    co2_mod = compute_co2(afr_mod, maf_mod, fuel)

    # 📏 Cálculo de métricas
    diff = np.abs(co2_orig - co2_mod)
    media = diff.mean()
    maxima = diff.max()
    rmse = mean_squared_error(co2_orig, co2_mod, squared=False)
    std = diff.std()

    print(f"\n🔁 {nome_mr} ({fuel.upper()})")
    print(f"Diferença Média: {media:.4f} | Diferença Máxima: {maxima:.4f} | Erro Quadrático Médio (RMSE): {rmse:.4f} | Desvio Padrão (Std): {std:.4f}")

    # 🧾 Tabela comparativa
    tabela = pd.concat([
        df_original[features].iloc[:10].add_suffix("_Orig"),
        pd.DataFrame({" ": ["→"] * 10}),
        df_modificado[features].iloc[:10].add_suffix("_Mod"),
        pd.DataFrame({" ": [" "] * 10}),
        pd.DataFrame({
            "CO₂_Real": co2_real[:10] if co2_real is not None else np.nan,
            "CO₂_Orig": co2_orig[:10],
            "CO₂_Mod": co2_mod[:10]
        })
    ], axis=1)

    display(tabela.style.set_caption(f"Comparação dos dados originais e modificados - {nome_mr} ({fuel.upper()})"))

    # 📊 Gráfico Plotly
    fig = go.Figure()
    if co2_real is not None:
        fig.add_trace(go.Scatter(y=co2_real[:100], mode='lines', name='Real', line=dict(dash='dot', color='green')))
    fig.add_trace(go.Scatter(y=co2_orig[:100], mode='lines', name='Predição Original', line=dict(dash='dash', color='blue')))
    fig.add_trace(go.Scatter(y=co2_mod[:100], mode='lines', name='Predição com RM', line=dict(color='red')))

    fig.update_layout(
        title=dict(text=f"Comparação CO₂ - {nome_mr} - {fuel.upper()}", x=0.5),
        xaxis_title="Amostras",
        yaxis_title="CO₂ (g/s)",
        template="plotly_white"
    )
    fig.show()

In [2]:
# 🔁 Testes Metamórficos
#INDICAR PARA CADA CT QUAL O TIPO DE RELACAO METAMORFICA
testes_metamorficos = {

# Scenario: RM1 - Velocidade nula
#Tipo de relação metamórfica: Exclusiva
    #     Given um trajeto real com velocidade variável
    #     When a velocidade é zerada em todos os pontos
    #     Then o CO₂ previsto deve refletir um cenário de motor parado, mesmo que a aceleração e posição geográfica estejam presentes
    "Caso de teste: RM1: Velocidade igual a zero (mantendo aceleração e posição)": lambda df: df.copy().assign(
        **{
            "Speed(OBD)(km/h)": 0.0
        }
    ),

# Scenario: RM2 - Latitude nula
#Tipo de relação metamórfica: Inversiva
    #     Given um trajeto com latitude geográfica real
    #     When a latitude é zerada em todos os pontos
    #     Then o CO₂ previsto deve manter coerência, a menos que o modelo dependa fortemente da posição geográfica
    "Caso de teste: RM2 - Latitude nula (mantendo demais variáveis)": lambda df: df.copy().assign(
        Latitude=0.0
    ),

    # Scenario: RM3 - Longitude nula
    #Tipo de relação metamórfica: Inversiva
    #     Given um trajeto com longitude geográfica real
    #     When a longitude é zerada em todos os pontos
    #     Then o CO₂ previsto deve manter coerência, a menos que o modelo dependa fortemente da posição geográfica
    "Caso de teste: RM3 - Longitude nula (mantendo demais variáveis)": lambda df: df.copy().assign(
        Longitude=0.0
    ),

    # Scenario: RM4 - Aceleração nula
    #Tipo de relação metamórfica: Exclusiva
    #     Given um trajeto com aceleração variável
    #     When a aceleração é zerada em todos os pontos
    #     Then o CO₂ previsto deve refletir comportamento compatível com velocidade constante ou baixa dinâmica
    "Caso de teste: RM4 - Aceleração nula (mantendo demais variáveis)": lambda df: df.copy().assign(
        Acceleration=0.0
    ),


# Scenario: RM5 - Velocidade constante no valor médio do trajeto
#Tipo de relação metamórfica: Aditiva
#    Given um trajeto com velocidade variável
#    When a velocidade é substituída pela média do trajeto
#    Then o CO₂ previsto deve se manter coerente e levemente suavizado

    "Caso de teste: RM5 - Velocidade constante": lambda df: df.assign(
        **{"Speed(OBD)(km/h)": df["Speed(OBD)(km/h)"].mean()}
    ),

# Scenario: RM6 - Velocidade constante em 80 km/h e aceleração zero
#Tipo de relação metamórfica: Aditiva + Exclusiva (A/E)
#     Given um trajeto urbano
#     When a velocidade é definida como 80 km/h e a aceleração é zero
#     Then o modelo deve prever um CO₂ constante ou levemente crescente

    "Caso de teste: RM6 - Velocidade constante em 80 km/h e aceleração igual a zero": lambda df: df.assign(
        **{
            "Speed(OBD)(km/h)": 80.0,
            "Acceleration": 0.0
        }        
    ),

# Scenario: RM7 - Velocidade constante a 130 km/h e aceleração zero
#Tipo de relação metamórfica: Aditiva + Exclusiva (A/E)
#     Given um trajeto urbano
#     When a velocidade é definida como 130 km/h e a aceleração é zero
#     Then o modelo deve prever um CO₂ elevado e estável

    "Caso de teste: RM7 - Velocidade constante a 130 km/h e aceleração igual a zero": lambda df: df.assign(
        **{
            "Speed(OBD)(km/h)": 130.0,
            "Acceleration": 0.0
        }
    ),

# Scenario: RM8 - Velocidade crescente linearmente (2 km/h por ponto)
#Tipo de relação metamórfica: Sustentável
#     Given um trajeto inicial com velocidade arbitrária
#     When a velocidade aumenta de 2 em 2 km/h e a aceleração é ajustada
#     Then o CO₂ previsto deve aumentar progressivamente de forma coerente

    "Caso de teste: RM8 - Velocidade crescente linearmente (2 km/h por ponto)": lambda df: (
        lambda speeds_kmh: df.assign(
            **{
                "Speed(OBD)(km/h)": speeds_kmh,
                "Acceleration": np.diff(
                    speeds_kmh * 1000 / 3600,  # conversão para m/s
                    prepend=speeds_kmh[0] * 1000 / 3600
                )
            }
        )
    )(np.arange(0, 2 * len(df), 2)),

    

# Scenario: RM9 - Latitude e Longitude constantes
#Tipo de relação metamórfica: Inversiva
#   Given um trajeto com diferentes coordenadas geográficas
#   When todas as posições são fixadas no ponto médio
#   Then o CO₂ previsto deve mudar pouco, indicando baixa dependência espacial

    "Caso de teste: RM9 - Latitude e Longitude constantes": lambda df: df.assign(
        Latitude=df["Latitude"].mean(),
        Longitude=df["Longitude"].mean()
    ),

# Scenario: RM10 - Aceleração constante
#Tipo de relação metamórfica: Aditiva
#     Given um trajeto com aceleração variável
#     When a aceleração é substituída pelo valor médio
#     Then o modelo deve produzir previsões de CO₂ similares ao trajeto original

    "Caso de teste: RM10 - Aceleração constante": lambda df: df.assign(
        Acceleration=df["Acceleration"].mean()
    ),


# Scenario: RM11 - Velocidade e aceleração iguais a zero
#Tipo de relação metamórfica: Exclusiva
#     Given um trajeto ativo
#     When todas as velocidades e acelerações são zeradas
#     Then o CO₂ previsto deve ser mínimo ou nulo, respeitando coerência física

    "Caso de teste: RM11 - Velocidade e aceleração iguais a zero": lambda df: df.assign(
        **{
            "Speed(OBD)(km/h)": 0.0,
            "Acceleration": 0.0
        }
    ),


# Scenario: RM12 - Latitude e Longitude multiplicadas por 2
#Tipo de relação metamórfica: Multiplicativa
#     Given um trajeto com coordenadas reais
#     When os valores de latitude e longitude são multiplicados por 2
#     Then o modelo pode apresentar instabilidades ou previsões incoerentes

    "Caso de teste: RM12 - Latitude e Longitude multiplicadas por 2": lambda df: df.assign(
        Latitude=df["Latitude"] * 2,
        Longitude=df["Longitude"] * 2
    ),

# Scenario: RM13 - Latitude e Longitude deslocadas para simular +200 km
#Tipo de relação metamórfica: Multiplicativa, Inversiva e Exclusiva
#     Given um trajeto típico
#     When a dispersão espacial das coordenadas é aumentada em 25 vezes
#     Then o modelo deve manter coerência ou indicar limitação fora da distribuição de treino

    "Caso de teste: RM13 - Latitude e Longitude deslocadas para simular +200 km": lambda df: df.assign(
        Latitude=(df["Latitude"] - df["Latitude"].mean()) * 25 + df["Latitude"].mean(),
        Longitude=(df["Longitude"] - df["Longitude"].mean()) * 25 + df["Longitude"].mean()
    ),
    
}


In [3]:
# 🧲 Executar para ambos os combustíveis e RMs
for nome_mr, transformacao in testes_metamorficos.items():
    for combustivel, df_original in [("ethanol", df_ethanol), ("gasoline", df_gasoline)]:
        df_modificado = transformacao(df_original.copy())
        executar_teste_plotly(
            df_original=df_original,
            df_modificado=df_modificado,
            modelo_afr=modelos[combustivel]["afr"],
            modelo_maf=modelos[combustivel]["maf"],
            fuel=combustivel,
            nome_mr=nome_mr
        )


🔁 Caso de teste: RM1: Velocidade igual a zero (mantendo aceleração e posição) (ETHANOL)
Diferença Média: 0.3016 | Diferença Máxima: 2.2210 | Erro Quadrático Médio (RMSE): 0.4900 | Desvio Padrão (Std): 0.3862


,Latitude_Orig,Longitude_Orig,Speed(OBD)(km/h)_Orig,Acceleration_Orig,,Latitude_Mod,Longitude_Mod,Speed(OBD)(km/h)_Mod,Acceleration_Mod,,CO₂_Real,CO₂_Orig,CO₂_Mod
0,-5.820105,-35.220818,3,0.000000,→,-5.820105,-35.220818,0.000000,0.000000,,0.642839,0.742631,0.727468
1,-5.820103,-35.220828,5,0.555556,→,-5.820103,-35.220828,0.000000,0.555556,,0.653249,0.771505,0.727468
2,-5.820102,-35.220838,5,0.000000,→,-5.820102,-35.220838,0.000000,0.000000,,0.653249,0.771505,0.727468
3,-5.820096,-35.220843,3,-0.555556,→,-5.820096,-35.220843,0.000000,-0.555556,,0.679275,0.659013,0.721582
4,-5.820091,-35.220850,0,-0.833333,→,-5.820091,-35.220850,0.000000,-0.833333,,0.610306,0.667689,0.667689
5,-5.820091,-35.220850,0,0.000000,→,-5.820091,-35.220850,0.000000,0.000000,,0.610306,0.706858,0.706858
6,-5.820091,-35.220850,0,0.000000,→,-5.820091,-35.220850,0.000000,0.000000,,0.490587,0.706858,0.706858
7,-5.820085,-35.220856,3,0.833333,→,-5.820085,-35.220856,0.000000,0.833333,,1.090483,1.137803,0.706858
8,-5.820074,-35.220859,3,0.000000,→,-5.820074,-35.220859,0.000000,0.000000,,1.090483,1.137803,0.706858
9,-5.820064,-35.220858,3,0.000000,→,-5.820064,-35.220858,0.000000,0.000000,,0.921315,1.137803,0.706858



🔁 Caso de teste: RM1: Velocidade igual a zero (mantendo aceleração e posição) (GASOLINE)
Diferença Média: 0.5511 | Diferença Máxima: 2.9554 | Erro Quadrático Médio (RMSE): 0.8179 | Desvio Padrão (Std): 0.6044


,Latitude_Orig,Longitude_Orig,Speed(OBD)(km/h)_Orig,Acceleration_Orig,,Latitude_Mod,Longitude_Mod,Speed(OBD)(km/h)_Mod,Acceleration_Mod,,CO₂_Real,CO₂_Orig,CO₂_Mod
0,-5.820010,-35.220825,5,0.000000,→,-5.820010,-35.220825,0.000000,0.000000,,1.002671,1.456781,0.965052
1,-5.820013,-35.220822,4,-0.277778,→,-5.820013,-35.220822,0.000000,-0.277778,,1.013360,1.016708,0.884055
2,-5.820016,-35.220819,5,0.277778,→,-5.820016,-35.220819,0.000000,0.277778,,2.088720,1.456781,0.965052
3,-5.820014,-35.220825,7,0.555556,→,-5.820014,-35.220825,0.000000,0.555556,,2.261889,1.996380,0.965052
4,-5.819992,-35.220829,9,0.555556,→,-5.819992,-35.220829,0.000000,0.555556,,2.499195,2.400486,0.940914
5,-5.819974,-35.220824,12,0.833333,→,-5.819974,-35.220824,0.000000,0.833333,,2.606089,2.400486,0.940914
6,-5.819944,-35.220823,15,0.833333,→,-5.819944,-35.220823,0.000000,0.833333,,3.070012,2.148721,0.880813
7,-5.819908,-35.220810,18,0.833333,→,-5.819908,-35.220810,0.000000,0.833333,,2.445747,1.470940,0.773157
8,-5.819866,-35.220794,18,0.000000,→,-5.819866,-35.220794,0.000000,0.000000,,1.073221,1.588029,0.783680
9,-5.819821,-35.220775,16,-0.555556,→,-5.819821,-35.220775,0.000000,-0.555556,,0.562265,1.085921,0.980167



🔁 Caso de teste: RM2 - Latitude nula (mantendo demais variáveis) (ETHANOL)
Diferença Média: 0.2759 | Diferença Máxima: 2.0479 | Erro Quadrático Médio (RMSE): 0.3941 | Desvio Padrão (Std): 0.2815


...

,Latitude_Orig,Longitude_Orig,Speed(OBD)(km/h)_Orig,Acceleration_Orig,,Latitude_Mod,Longitude_Mod,Speed(OBD)(km/h)_Mod,Acceleration_Mod,,CO₂_Real,CO₂_Orig,CO₂_Mod
0,-5.820010,-35.220825,5,0.000000,→,-11.640021,-70.441650,5,0.000000,,1.002671,1.456781,1.655048
1,-5.820013,-35.220822,4,-0.277778,→,-11.640026,-70.441645,4,-0.277778,,1.013360,1.016708,1.687906
2,-5.820016,-35.220819,5,0.277778,→,-11.640032,-70.441639,5,0.277778,,2.088720,1.456781,1.790437
3,-5.820014,-35.220825,7,0.555556,→,-11.640027,-70.441649,7,0.555556,,2.261889,1.996380,1.977492
4,-5.819992,-35.220829,9,0.555556,→,-11.639984,-70.441658,9,0.555556,,2.499195,2.400486,2.086244
5,-5.819974,-35.220824,12,0.833333,→,-11.639947,-70.441648,12,0.833333,,2.606089,2.400486,2.127853
6,-5.819944,-35.220823,15,0.833333,→,-11.639888,-70.441646,15,0.833333,,3.070012,2.148721,2.213368
7,-5.819908,-35.220810,18,0.833333,→,-11.639817,-70.441621,18,0.833333,,2.445747,1.470940,2.134576
8,-5.819866,-35.220794,18,0.000000,→,-11.639731,-70.441588,18,0.000000,,1.073221,1.588029,1.783583
9,-5.819821,-35.220775,16,-0.555556,→,-11.639642,-70.441549,16,-0.555556,,0.562265,1.085921,1.900804



🔁 Caso de teste: RM13 - Latitude e Longitude deslocadas para simular +200 km (ETHANOL)
Diferença Média: 0.3695 | Diferença Máxima: 2.4156 | Erro Quadrático Médio (RMSE): 0.4900 | Desvio Padrão (Std): 0.3218


,Latitude_Orig,Longitude_Orig,Speed(OBD)(km/h)_Orig,Acceleration_Orig,,Latitude_Mod,Longitude_Mod,Speed(OBD)(km/h)_Mod,Acceleration_Mod,,CO₂_Real,CO₂_Orig,CO₂_Mod
0,-5.820105,-35.220818,3,0.000000,→,-6.085970,-35.529517,3,0.000000,,0.642839,0.742631,0.989485
1,-5.820103,-35.220828,5,0.555556,→,-6.085921,-35.529767,5,0.555556,,0.653249,0.771505,0.905773
2,-5.820102,-35.220838,5,0.000000,→,-6.085899,-35.530011,5,0.000000,,0.653249,0.771505,0.888538
3,-5.820096,-35.220843,3,-0.555556,→,-6.085743,-35.530146,3,-0.555556,,0.679275,0.659013,0.890532
4,-5.820091,-35.220850,0,-0.833333,→,-6.085620,-35.530325,0,-0.833333,,0.610306,0.667689,0.994262
5,-5.820091,-35.220850,0,0.000000,→,-6.085620,-35.530325,0,0.000000,,0.610306,0.706858,0.938908
6,-5.820091,-35.220850,0,0.000000,→,-6.085620,-35.530325,0,0.000000,,0.490587,0.706858,0.938908
7,-5.820085,-35.220856,3,0.833333,→,-6.085479,-35.530462,3,0.833333,,1.090483,1.137803,1.008678
8,-5.820074,-35.220859,3,0.000000,→,-6.085202,-35.530556,3,0.000000,,1.090483,1.137803,0.989485
9,-5.820064,-35.220858,3,0.000000,→,-6.084938,-35.530517,3,0.000000,,0.921315,1.137803,0.989485



🔁 Caso de teste: RM13 - Latitude e Longitude deslocadas para simular +200 km (GASOLINE)
Diferença Média: 0.8312 | Diferença Máxima: 4.0360 | Erro Quadrático Médio (RMSE): 1.0172 | Desvio Padrão (Std): 0.5863


,Latitude_Orig,Longitude_Orig,Speed(OBD)(km/h)_Orig,Acceleration_Orig,,Latitude_Mod,Longitude_Mod,Speed(OBD)(km/h)_Mod,Acceleration_Mod,,CO₂_Real,CO₂_Orig,CO₂_Mod
0,-5.820010,-35.220825,5,0.000000,→,-6.002699,-35.508900,5,0.000000,,1.002671,1.456781,1.655048
1,-5.820013,-35.220822,4,-0.277778,→,-6.002767,-35.508839,4,-0.277778,,1.013360,1.016708,1.687906
2,-5.820016,-35.220819,5,0.277778,→,-6.002843,-35.508762,5,0.277778,,2.088720,1.456781,1.790437
3,-5.820014,-35.220825,7,0.555556,→,-6.002782,-35.508892,7,0.555556,,2.261889,1.996380,1.977492
4,-5.819992,-35.220829,9,0.555556,→,-6.002236,-35.508996,9,0.555556,,2.499195,2.400486,2.086244
5,-5.819974,-35.220824,12,0.833333,→,-6.001778,-35.508882,12,0.833333,,2.606089,2.400486,2.127853
6,-5.819944,-35.220823,15,0.833333,→,-6.001040,-35.508854,15,0.833333,,3.070012,2.148721,2.213368
7,-5.819908,-35.220810,18,0.833333,→,-6.000148,-35.508536,18,0.833333,,2.445747,1.470940,2.134576
8,-5.819866,-35.220794,18,0.000000,→,-5.999081,-35.508129,18,0.000000,,1.073221,1.588029,1.783583
9,-5.819821,-35.220775,16,-0.555556,→,-5.997965,-35.507640,16,-0.555556,,0.562265,1.085921,1.900804
